In [1]:

# import
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings

from langchain import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate
from langchain.llms.base import LLM
from transformers import AutoTokenizer, AutoModel, AutoConfig
from typing import Any, Dict, List, Mapping, Optional, Tuple, Union
from torch.mps import empty_cache
import torch
from langchain.chains import RetrievalQA

from langchain.document_loaders import UnstructuredFileLoader
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain import OpenAI
from langchain.docstore.document import Document  # Ensure this is the correct import for your document class


/home/dxw/anaconda3/envs/agent/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class GLM(LLM):
    max_token: int = 2048
    temperature: float = 0.8
    top_p = 0.9
    tokenizer: object = None
    model: object = None
    history_len: int = 1024
    
    def __init__(self):
        super().__init__()
        
    @property
    def _llm_type(self) -> str:
        return "GLM"
            
    def load_model(self, llm_device="gpu",model_name_or_path=None):
        model_config = AutoConfig.from_pretrained(model_name_or_path, trust_remote_code=True)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,trust_remote_code=True)
        self.model = AutoModel.from_pretrained(model_name_or_path, config=model_config, trust_remote_code=True, device='cuda:5').half() # GLM模块装在gpu: 6



    def _call(self,prompt:str,history:List[str] = [],stop: Optional[List[str]] = None):
        response, _ = self.model.chat(
                    self.tokenizer,prompt,
                    history=history[-self.history_len:] if self.history_len > 0 else [],
                    max_length=self.max_token,temperature=self.temperature,
                    top_p=self.top_p)
        return response

In [3]:
import sys
modelpath = "/data1/dxw_data/llm/chatglm3-6b-128k"
sys.path.append(modelpath)
llm = GLM()
llm.load_model(model_name_or_path = modelpath)
#---------------------------至此, 成功加载模型

2024-06-06 10:58:15.389276: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 10:58:15.541544: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-06 10:58:16.118477: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64
2024-06-06 10:58:16.118541: W tensorflow/compiler/xla/stream_exec

In [ ]:
# ----------------------------使用RAG利用数据库检索增强------------------------ #

In [4]:
import os
from langchain.docstore.document import Document  # Ensure this is the correct import for your document class
# Create prompt template
prompt_template = """请总结用户的评论有哪些主题关键词，请直接输出关键词:


TEXT:
{text}

"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

# Load the summarization chain
# chain = load_summarize_chain(llm, chain_type="stuff", return_intermediate_steps=True, map_prompt=PROMPT, combine_prompt=PROMPT)
chain = load_summarize_chain(llm, chain_type="stuff", prompt=PROMPT)

# Initialize the text splitter
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)

# Create the embedding function
model_kwargs = {'device': 'cuda:6'}
embedding_function = HuggingFaceEmbeddings(model_name='/data1/dxw_data/llm/text2vec-large-chinese', model_kwargs=model_kwargs)

# Directory containing the text files
input_dir = '/data1/dxw_data/llm/MKT_data_mining/DB/document_ans/input.txt'
output_prefix = 'output_'

No sentence-transformers model found with name /data1/dxw_data/llm/text2vec-large-chinese. Creating a new one with MEAN pooling.


In [6]:
# List to store the summarized keywords
keywords_list = []

# Loop to read the contents of each text file and summarize the theme keywords
for i in range(1, 100):  # Adjust the range as necessary
    file_path = os.path.join(input_dir, f'{output_prefix}{i}.txt')
    if not os.path.exists(file_path):
        break
    
    print("file_path: ",file_path)
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
        
    # Prepare the document for summarization
    documents = [Document(page_content=text)]
    
    # Split the document into chunks
    docs = text_splitter.split_documents(documents)
    
    # Summarize the theme keywords
    result = chain({"input_documents": docs}, return_only_outputs=True)
    print(result)
    
    # Assuming result contains the summarized keywords
    keywords = result.get('output_text', '')  # Adjust based on the actual key in the result
    
    # Add the keywords to the list
    keywords_list.append(keywords)

# Print the list of keywords
print(keywords_list)

# Optionally, save the keywords list to a file
keywords_output_path = '/data1/dxw_data/llm/mkt_llm/mkt_medicine/keywords_list.txt'
with open(keywords_output_path, 'w', encoding='utf-8') as f:
    for keywords in keywords_list:
        f.write(f"{keywords}\n")

file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_1.txt
{'output_text': '关键词：脑部病灶、检查结果、化疗、靶向四期、脑部病灶反复出现、放疗、伽马刀、质子放疗、肺部放疗、担心肺炎、呕吐、严重症状。'}
file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_2.txt
{'output_text': '关键词：\n1. 生命\n2. 过程\n3. 经历\n4. 珍惜时间\n5. 成就更好的自己\n6. 晨光\n7. 期待\n8. 步履不停\n9. 精彩\n10. 充实\n11. 高考\n12. 基因突变\n13. 治疗效果\n14. 化学药物\n15. 副作用\n16. 皮疹\n17. 保持皮肤清洁\n18. 勤洗澡\n19. 换衣\n20. 皮炎平\n21. 炉甘石洗剂\n22. 氯雷他定\n23. 抓挠\n24. 感染\n25. 碘伏\n26. 红霉素\n27. 百多邦\n28. 美满霉素\n29. 饮食清淡\n30. 麻辣食品'}
file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_3.txt
{'output_text': '1. 肿瘤诊断\n2. 生活态度\n3. 家庭关系\n4. 心理状态\n5. 疾病治疗\n6. 父母关爱\n7. 生活琐事\n8. 生活压力\n9. 家庭责任'}
file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_4.txt
{'output_text': '关键词：\n\n1. 互相欣赏\n2. 争短长\n3. 沉默\n4. 口舌之争\n5. 修养\n6. 息事宁人\n7. 成全\n8. 成年人世界\n9. 肺部薄层高分辨CT\n10. 主治医师\n11. 肿瘤\n12. 确诊\n13. 治疗方案\n14. 专家建议\n15. 病灶持续缩小\n16. 阳性结果\n17. 高兴与悲哀\n18. 治疗征程\n19. 肿瘤晚期\n20. 肺内多发转移\n21. 骨转移\n22. 建。'}
file_pat

Created a chunk of size 578, which is longer than the specified 100
Created a chunk of size 145, which is longer than the specified 100
Created a chunk of size 777, which is longer than the specified 100
Created a chunk of size 168, which is longer than the specified 100


{'output_text': '主题关键词：肿瘤标志物、复查、肺癌、治疗、家族遗传史、碎片时间、医学基础、经验、化疗、眼眶痛、痰培养、咳嗽、出痰、胸部疼痛、睡眠质量、贝伐、阿美替尼、焦虑、晚霞。'}
file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_6.txt


/home/dxw/anaconda3/envs/agent/lib/python3.8/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 2176, but `max_length` is set to 2048. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


{'output_text': ''}
file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_7.txt


...

{'output_text': ''}
file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_93.txt
{'output_text': '根据用户评论的内容，我们可以总结出以下主题关键词：\n\n1. 肺癌：评论中有多位患者提到了自己或家人患有肺癌。\n2. 治疗方案：评论中讨论了各种肺癌治疗方案，包括手术、化疗、靶向治疗、免疫治疗等。\n3. 复发：有多位患者提到了肺癌复发的可能性，以及复发后的治疗方案。\n4. 基因检测：多位患者提到了进行基因检测，以确定肺癌的分子分型和治疗方案。\n5. 饮食：有患者在讨论肺癌患者的饮食建议。\n6. 睡眠：有一位患者提到了肺癌治疗对睡眠的影响。\n7. 咳嗽：有多位患者提到肺癌治疗后的咳嗽症状。\n8. 声音嘶哑：有一位患者提到了肺癌治疗后的声音嘶哑症状。\n9. 副作用：多位患者提到了肺癌治疗中的副作用。\n10. 咨询：有多位患者在寻求其他患者的意见和建议。'}
file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_94.txt
{'output_text': '关键词：生活、快乐、充实、健康、饮食、睡眠、亲情、亲情、感恩、忙碌、聊天、散步、公园、自行车、超市、买菜、压力锅、红苋菜、手机、刷刷、午饭、视频、消食、午觉、电话、聊天、散步、跳交谊舞、吸氧、洗澡、内衣裤、洗澡巾、洗头洗澡、珊宝、妈妈、按摩、床上、睡觉、晨光、期待、步履不停、新的一天、早安。'}
file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_95.txt
{'output_text': '关键词:\n\n1. 出国医疗\n2. 病假\n3. 照顾病人\n4. 老人\n5. 洗头洗澡\n6. 久病床前无孝子\n7. 梦想\n8. 现实\n9. 理想\n10. 山路\n11. 星辰'}
file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_96.txt
{'output_text': '用户评论的主要主题关键词有：加油、好好吃饭、相信奇迹、许愿、复

/home/dxw/anaconda3/envs/agent/lib/python3.8/site-packages/transformers/generation/utils.py:1268: UserWarning: Input length of input_ids is 2604, but `max_length` is set to 2048. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


{'output_text': ''}
file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_98.txt


Created a chunk of size 469, which is longer than the specified 100
Created a chunk of size 122, which is longer than the specified 100


{'output_text': '1. 培美\n2. 贝伐\n3. CT\n4. 白紫\n5. 卡铂\n6. 用药\n7. 效果\n8. 淋巴结\n9. 空窗期\n10. 身体伤害'}
file_path:  /data1/dxw_data/llm/mkt_llm/mkt_medicine/data2/output_99.txt
{'output_text': '关键词总结如下：\n\n1. 交通便捷：孩子回家时间缩短\n2. 父母饮食起居：照顾父母\n3. 照顾病人：照顾生病父母\n4. 肿瘤复查：肿瘤复查经验分享\n5. 营养不良：营养不良问题\n6. 心理状态：心理状态的调整\n7. 医疗治疗：医疗治疗经验分享\n8. 身体健康：身体健康状况关注'}
['关键词：脑部病灶、检查结果、化疗、靶向四期、脑部病灶反复出现、放疗、伽马刀、质子放疗、肺部放疗、担心肺炎、呕吐、严重症状。', '关键词：\n1. 生命\n2. 过程\n3. 经历\n4. 珍惜时间\n5. 成就更好的自己\n6. 晨光\n7. 期待\n8. 步履不停\n9. 精彩\n10. 充实\n11. 高考\n12. 基因突变\n13. 治疗效果\n14. 化学药物\n15. 副作用\n16. 皮疹\n17. 保持皮肤清洁\n18. 勤洗澡\n19. 换衣\n20. 皮炎平\n21. 炉甘石洗剂\n22. 氯雷他定\n23. 抓挠\n24. 感染\n25. 碘伏\n26. 红霉素\n27. 百多邦\n28. 美满霉素\n29. 饮食清淡\n30. 麻辣食品', '1. 肿瘤诊断\n2. 生活态度\n3. 家庭关系\n4. 心理状态\n5. 疾病治疗\n6. 父母关爱\n7. 生活琐事\n8. 生活压力\n9. 家庭责任', '关键词：\n\n1. 互相欣赏\n2. 争短长\n3. 沉默\n4. 口舌之争\n5. 修养\n6. 息事宁人\n7. 成全\n8. 成年人世界\n9. 肺部薄层高分辨CT\n10. 主治医师\n11. 肿瘤\n12. 确诊\n13. 治疗方案\n14. 专家建议\n15. 病灶持续缩小\n16. 阳性结果\n17. 高兴与悲哀\n18. 治疗征程\n19. 肿瘤晚期\n20. 肺内多发转移\n21. 骨转移\n22. 建。', '主题关键词：肿瘤标志物、复查、肺癌、治疗、家族遗传